In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import credit_card_data

import torch.nn as nn
import torch
import torch.nn.functional as F
from torchsummary import summary

import matplotlib.pyplot as plt

from tqdm import tqdm
from random import shuffle

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
window_size = 15

Z,Y = credit_card_data.generate_raw_data(60, 10000)
train_data, train_label, test_data, test_label = credit_card_data.generate_win_data(Z, Y, window_size)
train_data = torch.tensor(train_data, dtype=torch.float)
train_label = torch.tensor(train_label, dtype=torch.float)
test_data = torch.tensor(test_data, dtype=torch.float)
test_label = torch.tensor(test_label, dtype=torch.float)

In [4]:
print(train_label.size())
print(train_label.sum())

torch.Size([49947])
tensor(3224.)


In [103]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.batch_size = batch_size
        self.cnn1 = nn.Sequential(
            nn.Conv1d(
                in_channels = 1,
                out_channels = 5,
                kernel_size = 3
            )
        )
        self.cnn2 = nn.Sequential(
            nn.Conv1d(
                in_channels = 5,
                out_channels = 5,
                kernel_size = 3
            )
        )
        self.cnn3 = nn.Sequential(
            nn.Conv1d(
                in_channels = 5,
                out_channels = 5,
                kernel_size = 3
            ))
        self.cnn4 = nn.Sequential(
            nn.Conv1d(
                in_channels = 5,
                out_channels = 5,
                kernel_size = 3
            ))
        self.linear = nn.Linear(5,1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input):
        x = self.cnn1(input);print(x.size())
        x = self.cnn2(x);print(x.size())
        x = self.cnn3(x);print(x.size())
        x = self.cnn4(x);print(x.size())
        x = torch.squeeze(x);print(x.size())
        linear_out = self.linear(x)
        y_pred = self.sigmoid(linear_out)
        return y_pred

In [100]:
batch_size = 512
model = CNN()

num_epochs = 3

lstm_input_size = window_size
h1 = 3
num_train = len(train_data)
output_dim = 1
num_layers = 1
# model = LSTM(lstm_input_size, h1, batch_size=batch_size, output_dim=output_dim, num_layers=num_layers)

model.to(device)
learning_rate = 0.001
loss_fn = torch.nn.BCELoss(size_average=False)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [101]:
len(train_data)//batch_size

97

In [102]:
model = CNN()
model.to(device)

for epoch in range(50):
    running_loss = 0.0
    train = np.concatenate([train_data,train_label.unsqueeze(1)],axis = 1)
    shuffle(train)
    train_data_ = torch.tensor(train[:,:12], device=device)
    train_label_ = torch.tensor(train[:,12], device=device)
    for i in range(len(train_data)//batch_size):
#         optimizer.zero_grad()
        batch_data = train_data_[i*batch_size:(i+1)*batch_size].unsqueeze(-1).permute(0,2,1).to(device)
        batch_label = train_label_[i*batch_size:(i+1)*batch_size].to(device)
        y_pred = model(batch_data)
        weight = 10 * (batch_label == 1).float() + torch.ones(batch_size, device=device)
        loss = torch.nn.BCELoss(weight=weight)(y_pred, batch_label)
        loss.backward()
        optimiser.step()
        running_loss += loss.item()
        if i % 97 == 1:
            print('[%d, %5d] loss: %.15f' % (epoch + 1, i + 1, running_loss / 97))
            running_loss = 0.0

torch.Size([512, 5, 10])
torch.Size([512, 5, 8])
torch.Size([512, 5, 6])
torch.Size([512, 5, 5])
torch.Size([512, 5, 5])


D:\anaconda\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 5, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


ValueError: Target and input must have the same number of elements. target nelement (512) != input nelement (2560)

In [ ]:
def criteria(y_test, test_label):
    recall = []
    precision = []
    for t in range(102):
        TP = 0
        TN = 0
        FP = 0
        FN = 0
        threshold = t*0.01 - 0.01
        TP += ((y_test > threshold).numpy().flatten()&(test_label == 1).numpy()).sum()
        TN += ((y_test < threshold).numpy().flatten()&(test_label == 0).numpy()).sum()
        FP += ((y_test > threshold).numpy().flatten()&(test_label == 0).numpy()).sum()
        FN += ((y_test < threshold).numpy().flatten()&(test_label == 1).numpy()).sum()
        recall.append((TP) / (TP + FN))
        precision.append((TP) / (TP + FP))
    return recall, precision

In [ ]:
y_test = model(test_data.unsqueeze(-1).permute(0,2,1)).cpu()
recall, precision = criteria(y_test, test_label)
plt.plot(recall, precision)
plt.xlim(0,1)
plt.ylim(0,1)